In [ ]:
# import libraries
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

In [ ]:
names = ["class", "message"]
train_file = pd.read_csv(train_file_path, sep='\t', names=names)
test_file = pd.read_csv(test_file_path, sep='\t', names=names)

train_message = train_file["message"].values.tolist()
train_label = np.array([0 if x=="ham" else 1 for x in train_file['class'].values.tolist()])
test_message = test_file["message"].values.tolist()
test_label = np.array([0 if x=="ham" else 1 for x in test_file['class'].values.tolist()])

vocabulary_dict = {}
for message in train_message:
    for vocabulary in message.split():
        if vocabulary not in vocabulary_dict:
            vocabulary_dict[vocabulary] = 1
        else:
            vocabulary_dict[vocabulary] += 1

VOCAB_SIZE = len(vocabulary_dict)
MAX_LENGTH = len(max(train_message, key=lambda p: len(p.split())).split())

def encode_message(message, vocabulary_dict):
    encoded = []
    for word in message.split():
        if word in vocabulary_dict:
            encoded.append(vocabulary_dict[word])
    return encoded

encoded_train_message = [encode_message(message, vocabulary_dict) for message in train_message]
padded_train_message = np.zeros((len(encoded_train_message), MAX_LENGTH))
for i, encoded_message in enumerate(encoded_train_message):
    padded_train_message[i, :len(encoded_message)] = encoded_message

encoded_test_message = [encode_message(message, vocabulary_dict) for message in test_message]
padded_test_message = np.zeros((len(encoded_test_message), MAX_LENGTH))
for i, encoded_message in enumerate(encoded_test_message):
    padded_test_message[i, :len(encoded_message)] = encoded_message

In [ ]:
import tensorflow as tf
from tensorflow import keras
model = tf.keras.Sequential()
embedding_layer = tf.keras.layers.Embedding(input_dim=VOCAB_SIZE, output_dim=100, input_length=MAX_LENGTH)
model.add(embedding_layer)
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
monitor = tf.keras.callbacks.EarlyStopping(monitor='val_acc', min_delta=1e-4, patience=25, verbose=1, mode='max', restore_best_weights=True)
model.fit(padded_train_message, train_label, validation_data=(padded_test_message, test_label), callbacks=[monitor], epochs=50, verbose=2)


In [ ]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])


def predict_message(pred_text):
    class_dict = {0: "ham", 1: "spam"}
    encoded_message = [encode_message(pred_text, vocabulary_dict)]
    padded_message = np.zeros((len(encoded_message), MAX_LENGTH))
    for i, encoded_message in enumerate(encoded_message):
        padded_message[i, :len(encoded_message)] = encoded_message
    prediction = [model.predict(padded_message)[0][0], class_dict[np.round(model.predict(padded_message)[0][0])]]
    return prediction

pred_text = "how are you doing today?"
prediction = predict_message(pred_text)
print(prediction)

In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = False

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    print(ans)
    if prediction[1] != ans:
      passed = True

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()
